In [ ]:
from libraries import *
from dataclasses import dataclass
from funtions import get_portfolio_value


@dataclass
class Position:
    ticker: str
    n_shares: int
    price: float
    sl: float
    tp: float
    time: str
    side: str  

def backtest(data, capital, trial) -> float:

    data= data.copy()
    # Indicadores e Hyperparametros
    # -- RSI -- Es el momentum de sobrecompra y sobreventa
    # -- Momentum -- Mide la velocidad del cambio del precio
    # -- Volatilidad -- Mide la variación del precio en el tiempo

    # --- Hyperparámetros ---
    rsi_window = trial.suggest_int('rsi_window', 46, 50)
    rsi_lower = trial.suggest_int('rsi_lower', 45, 55)
    rsi_upper = trial.suggest_int('rsi_upper', 55, 70)

    momentum_window = trial.suggest_int('momentum_window', 5, 50)
    momentum_threshold = trial.suggest_float('momentum_threshold', 0, 10)

    volatility_window = trial.suggest_int('volatility_window', 5, 30)
    volatility_threshold = trial.suggest_float('volatility_threshold', 4, 7)

    stop_loss = trial.suggest_float('stop_loss', 0.01, 0.15)
    take_profit = trial.suggest_float('take_profit', 0.01, 0.15)
    n_shares = trial.suggest_int('n_shares', 50, 500)

    # --- Indicadores ---
    data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=rsi_window).rsi()
    data['Momentum'] = ta.momentum.ROCIndicator(data['Close'], window=momentum_window).roc()
    data['Volatility'] = ta.volatility.BollingerBands(data['Close'], window=volatility_window).bollinger_wband()

    # --- Señales ---
    data['buy_signal'] = ((data['RSI'] < rsi_lower) & 
                        (data['Momentum'] > momentum_threshold) &
                        (data['Volatility'] < volatility_threshold))

    data['sell_signal'] = ((data['RSI'] > rsi_upper) &
                        (data['Momentum'] < -momentum_threshold) &
                        (data['Volatility'] > volatility_threshold))
    
    # --- Backtest ---
    initial_capital = capital
    margin_account = 0.0
    portafolio_value = [capital]

    active_long_positions: list[Position] = []
    active_short_positions: list[Position] = []


    # Commisions and Rates
    COM: float = 0.125 / 100

    # Convert annualized borrow rate to 5-min bar rate
    bars_per_year = 252 * 6.5 * 60 / 5  # 252 trading days, 6.5 hours per day, 5-min bars
    
    # Strategy parameters
    STOP_LOSS: float = stop_loss
    TAKE_PROFIT: float = take_profit
    N_SHARES: int = n_shares
    


    #### BACKTEST CORREGIDO

    for i, row in data.iterrows():
        # -- LONG -- #
        for position in active_long_positions.copy():
            # Stop loss o Take profit
            if row.Close > position.tp or row.Close < position.sl:
                capital += row.Close * position.n_shares * (1 - COM)
                active_long_positions.remove(position)

        # -- SHORT -- #
        for position in active_short_positions.copy():
            # Stop loss o Take profit
            if row.Close < position.tp or row.Close > position.sl:
                # Ganancia/Pérdida de la posición short
                cover_cost = row.Close * position.n_shares * (1 + COM)
                capital += (position.price - row.Close) * position.n_shares - cover_cost
                active_short_positions.remove(position)

        # --- Entradas --- #
        # Long
        if getattr(row, "buy_signal", False):
            cost = row.Close * N_SHARES * (1 + COM)
            if capital > cost:
                capital -= cost
                pos = Position(
                    ticker='BTCUSDT', n_shares=N_SHARES, price=row.Close,
                    sl=row.Close * (1 - STOP_LOSS), tp=row.Close * (1 + TAKE_PROFIT),
                    time=row.Datetime
                )
                active_long_positions.append(pos)
                # Debug: print("Se abrió LONG:", row.Datetime, row.Close)

        # Short
        if getattr(row, "sell_signal", False):
            # En short vendemos primero
            entry_cash = row.Close * N_SHARES * (1 - COM)
            capital += entry_cash
            pos = Position(
                ticker='BTCUSDT', n_shares=N_SHARES, price=row.Close,
                sl=row.Close * (1 + STOP_LOSS), tp=row.Close * (1 - TAKE_PROFIT),
                time=row.Datetime
            )
            active_short_positions.append(pos)
            # Debug: print("Se abrió SHORT:", row.Datetime, row.Close)

        # Portfolio value
        portafolio_value.append(
            get_portfolio_value(capital, active_long_positions, active_short_positions, row.Close, N_SHARES, COM)
        )

    # --- Retorno final del backtest ---
    return float(capital / 1_000_000 - 1)

